In [1]:
import csv
import sys
import numpy as np
import math
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
from math import sqrt


import xgboost as xgb

d = pd.read_csv("raw_open.csv",index_col=False)

In [2]:
def pega(sentence):
    sent_str = ""
    for i in sentence:
        sent_str += str(i)
    return sent_str
def cat(outfilename, line):
    with open(outfilename, 'w') as outfile:
            outfile.write(line)
            outfile.close()
def cat2(outfilename, line):
    with open(outfilename,'a') as outfile:
            outfile.write(line)
            outfile.close()

In [3]:
codigo =9
prueba_numero =2

f = 30
p = 20

prefix ='C:/Users/Celia/Desktop/4º/TFG/codigo/python/resultados'
path = pega([prefix,'/errorNaive',codigo,prueba_numero,'.txt'])

cat(path,pega(["p: ",p," f: ",f,"\n"]))
errores =pega([prefix,'/rmse',codigo,prueba_numero,'.txt'])
cat2(errores,pega(["Rmse mejores",prueba_numero,"\n"]))
#archivo registro de pruebas
n=pega([prefix,'/registro_pruebas',codigo,'.txt'])
cat2(n,pega(['nº',prueba_numero ,' columna_predecir: BDIY_Index_Open ',"p: ",p," f: ",f," columnas: ",len(d.columns)," n: ",3,"\n"]))


max_col = 3

In [ ]:
import warnings
warnings.filterwarnings('ignore')
for col in list(d.columns.values):
    #print(col)
    if(col=='BDIY_Index_Open'):# probar solo con euro
        
        present = d[(p):(len(d)-f)][col].reset_index()
        future = d[(f+p):len(d)][col].reset_index()
        # la etiqueta es el % de incremento en f días (saltando los p+1 primeros)

        label1 = future.subtract(present, fill_value=0).reset_index()
        label2 = (label1[col]/present[col]).reset_index()
        label=label2.drop(label2.columns[[0]], axis='columns') 
    
        array = [0] * len(label)
        zeros = pd.DataFrame(data=array)

        a = label.values
       
        rmse =sqrt(mean_squared_error(array, a))
        cat2(path,pega([col," ", rmse,"\n"]))
        # hasta aqui columna label y error del naive
        
        rmse_up_to_now = 1
        columns= []
        column_names = []

        n = 0
        puntoFijo = False
        cambios = 0
        # miemtras que no se lleven el máximo de columnas y siga mejorando
        while n < max_col and (not puntoFijo):
            print(pega(["n=", n]))
            ndf = columns
            ini = 0
            fin = len(d.columns)
            cambios=0
            
            #recorre todas las columnas
            for i in np.arange(ini, fin):
                
                #nombre de la columna
                nombres = pega([list(d.columns.values)[i],2])
                #si esa columna no ha sido cogida ya antes
               
                if not (nombres in column_names):
                    print(i,list(d.columns.values)[i])
                    pred = i
                    s = np.arange(1, len(d)+1)# filas de la tabla
                    vs = np.arange(0, p-1) # pasados a utilizar es ahasta p-1 porque no incluyepe la funcion arange
                    name = list(d.columns.values)[pred]
                    names = []
                    #para la columna  i con la que quiero hacer la prediccion  en names estan los posibles pasados de esa columna a usar
                    for v in vs:
                        names.append(pega([name, (p-v)]))
                    df=pd.DataFrame(columns=names)
                    # desde el dia p+1(cosas de indices) hasta el final menos futuro --> tabla
                    for i in np.arange(p,len(d)-f):
                        #para cada filla nrwo representa la fila actual
                        nrow=[]
                        #rellenar las columnas de (EuroPasado10, EuroPasado9, EuroPadaso8....)
                        for v in vs:
                            #calcular los incrementos que ha habido  en un marco temporal de 10 dias atras
                            #d[ el dia a predeccir - (p-v= rango de 10 a 1) dias atras]
                            nrow.append(((d.iloc[i-(p-v)+1][pred])-(d.iloc[i-(p-v)][pred]))/ (d.iloc[i-(p-v)][pred]))
                        df.append(nrow)
                        df = df.append(pd.Series(nrow, index=df.columns ), ignore_index=True)
                     
                    
                    
                    # se añade el mejor conjunto de columnas de la iteracion anterior de n si no es vacio
                    if(len(columns)!=0):
                        df=pd.concat([columns, df], axis=1, sort=False)
                    
                    # crear la matriz de datos con label
                    datos = pd.concat([label, df], axis=1, sort=False)
                    
                   
                    
                    
                    #-----------------EVALUACION---------------------------------------
                   
                        
                            
                   
                    
                    trainsize=1500 # tamaño de inicion del conjunto de entrenamiento
                    step=10
                    s =np.arange(trainsize,len(datos)-f,step)
                   
                    ## nuevo---------------
                   
                    pred = datos[col]
                   
                    data =datos.drop(col, 1)
                    #----------------------------------------
                    predichos = []
                    reales=[]
                    for j in s:
                        train_x = data[1:j][:]
                        train_y = pred[1:j][:]
                        
                        rfr = RandomForestRegressor()
                    

                        
                        rfr.fit(np.array(train_x), np.array(train_y))
                        
                        test=data.iloc[[j+f],:]
                        Y_pred = rfr.predict(test)
                        real =pred[j+f]
                        
                        
                        valor_p =Y_pred[0]
                        predichos.append(valor_p)
                        valor_r=real
                        reales.append(valor_r)
                    
                    
                    rmse =sqrt(mean_squared_error(reales, predichos))
                    cambios +=1
                    
                    #si mejora
                    if(not(np.isnan(rmse)) and rmse<rmse_up_to_now):
                        cambios +=1
                        rmse_up_to_now=rmse
                        ndf= df
                        cat2(errores,pega([col, " con ", name," iteracion", n," ", rmse_up_to_now,"\n"]))
                
                        
             
            print(pega(["mejores columnas iteracion: ",n]))
            print(ndf.columns.values)
                  
            # aqui termina la busqueda de la combinacion de pasasdos para las columnas i para predecir la columna col         
            if(cambios>0 or len(columns==0) or not (columns.columns.values ==ndf.columns.values) ) :
               
                columns = ndf
              
                columns_names = columns.columns.values
                
                print("cambio columnas")
                print(column_names)
                n+=1
            else:
                print("vuelta sin cabios")
                puntoFijo = True
                    
        print("Acabado")
        cat2(errores,pega([col, " acabado ", ndf.columns.values, rmse_up_to_now,"\n"]))
        print(rmse_up_to_now)
        print(ndf.columns.values)

n=0
0 EUR_Curncy_Open
1 JPY_Curncy_Open
2 GBP_Curncy_Open
3 CHF_Curncy_Open
4 CAD_Curncy_Open
5 NOK_Curncy_Open
6 INDU_Index_Open
7 SPX_Index_Open
8 CCMP_Index_Open
9 SPTSX_Index_Open
10 MEXBOL_Index_Open
11 UKX_Index_Open
12 CAC_Index_Open
13 DAX_Index_Open
14 IBEX_Index_Open
15 FTSEMIB_Index_Open
16 OMX_Index_Open
17 NKY_Index_Open
18 HSI_Index_Open
19 SHSZ300_Index_Open
20 AS51_Index_Open
21 TY1_Comdty_Open
22 FV1_Comdty_Open
23 TU1_Comdty_Open
24 RX1_Comdty_Open
25 OE1_Comdty_Open
26 DU1_Comdty_Open
27 GSPG2YR_Index_Open
28 GSPG5YR_Index_Open
29 GSPG10YR_Index_Open
30 CO1_Comdty_Open
31 CL1_Comdty_Open
32 XAu_Comdty_Open
33 XAG_Comdty_Open
34 C_1_Open
35 w_1_Open
36 S_1_Open
37 MO1_Comdty_Open
38 BDIY_Index_Open
39 CRY_Index_Open
40 HG1_Comdty_Open
41 LMAHDS03_Comdty_Open
42 LMCADS03_Comdty_Open
mejores columnas iteracion: 0
['IBEX_Index_Open20' 'IBEX_Index_Open19' 'IBEX_Index_Open18'
 'IBEX_Index_Open17' 'IBEX_Index_Open16' 'IBEX_Index_Open15'
 'IBEX_Index_Open14' 'IBEX_Index_Open